In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def excel_concat(FOLDERPATH, head=None) -> pd.DataFrame:
    dataframe = pd.DataFrame()
    files     = os.listdir(FOLDERPATH)
    for file in files:
        path_ = f'{FOLDERPATH}\{file}'
        if os.path.isfile(path_):
            tmp = pd.read_excel(path_,header=head)
            tmp[9] = os.path.basename(path_)
            dataframe = pd.concat([dataframe, tmp], ignore_index=True)
        else:
            continue
    return dataframe

COMMON_FOLDER = r'\\soyuzopttorg\spb\Ароматизаторы'
SOURCE_FOLDER = r'\Цены SOT Offer 2020 from Radkevich )'
TECH_GUIDE    = r'\input\technology.xlsx'                               #Технологическая база знаний
EXCEL_GUIDE   = r'\запрос цен гаер 2018-новый макрос новый формат.xlsm'

PRICE_PATH    = f'{COMMON_FOLDER}{SOURCE_FOLDER}'
TECH_PATH     = f'{COMMON_FOLDER}{SOURCE_FOLDER}{TECH_GUIDE}'
EXCEL_PATH    = f'{COMMON_FOLDER}{EXCEL_GUIDE}'


akciz_limit   = 8     # граница взимания акциза (% алкоголя)
akciz         = 8     # акциз за литр безводного спирта (EUR)
delivery      = 0.44  # стоимость доставки на кг (EUR)
vat           = 1.22  # НДС + 1
k1            = 1.5   # наценка 1 колонки + 1
k2            = 1.4   # наценка 2 колонки + 1
k3            = 1.3   # наценка 3 колонки + 1
currency      = 'EUR' # валюта прайса

symrise       = ('Symrise AG, (Германия)', 
                 'Симрайз Рогово',
                 'Symrise UK Ltd. (Great Britain)')


##fee_df                      = pd.read_excel(EXCEL_PATH,header=3, sheet_name='Расчет акциза')

tech_guide_df               = pd.read_excel(TECH_PATH,header=0)
tech_guide_df               = tech_guide_df.loc[tech_guide_df['Производитель'].isin(symrise)]
#tech_guide_df               = tech_guide_df.loc[tech_guide_df['Ассорт'] != '06. Исключили из ассортимента']
tech_guide_df['Код_чистый'] = tech_guide_df['Код'].str.slice(start=0, stop=6)

cost_fields = tech_guide_df[
    ['Код',
     'Ассорт',
     'Наименование', 
     'Код_чистый', 
     'Пошлина РФ%', 
     'Спирт', 
     'Розн цена']
].fillna(0)


fca_df = excel_concat(PRICE_PATH)
#fca_df = fca_df.drop_duplicates(keep='first')

fca_df.columns = ['Code', 
                  'Flavor_name', 
                  'Code_clean', 
                  'Lot', 
                  'Measure_unit', 
                  'FCA_price',
                  'Currency', 
                  'Valid_since', 
                  'Valid_until',
                  'filename'
                 ]


fca_df['Code_clean'] = fca_df['Code'].str.slice(start=2, stop=8)                   # очищаем артикул от лишних символов
fca_df.loc[fca_df['Valid_until'].isnull(), 'Valid_until'] = fca_df['Valid_since']  # устраняем путаницу в датах

price_calc = fca_df.merge(cost_fields, 
                          how='outer', 
                          left_on='Code_clean', 
                          right_on='Код_чистый'
                         ).fillna(0)


price_calc['Акциз'] = (price_calc['Спирт'] * akciz / 0.8) / 100           # расчет акциза
price_calc.loc[price_calc['Спирт'] <= akciz_limit, 'Акциз'] = 0           # чистим от лишних расчетов

price_calc['Себестоимость'] = (                                           # считаем себестоимость
    (price_calc['FCA_price'] + delivery) 
    * ((100 + price_calc['Пошлина РФ%'])/100) 
    + price_calc['Акциз']) * vat

price_calc['1колонка'] = round(price_calc['Себестоимость'] * k1, 1)       # cчитаем колонки прайса
price_calc['2колонка'] = round(price_calc['Себестоимость'] * k2, 1)
price_calc['3колонка'] = round(price_calc['Себестоимость'] * k3, 1)

price_calc.loc[price_calc['Код']  == 0, 'Код'] = price_calc['Code_clean']  # проставляем код 
price_calc.loc[price_calc['Наименование'] == 0,                            # и наименование отсутствующим в базе позициям
               'Наименование'] = price_calc['Flavor_name']                

price_calc = price_calc.loc[price_calc['FCA_price'] > 0]                   # отфильтровываем посчитанные цены
#price_calc = price_calc.loc[price_calc['Ассорт'] != '10. Бонусная позиция']
price_calc['Currency'] = currency
#price_upload = price_calc.loc[price_calc['Розн цена'] == 0]                # отфильтровываем непрайсовые позиции

#price_calc = price_calc.drop_duplicates(keep='first')

columns_to_upload = price_calc[['Код',                                   # готовим файл для загрузки
                                  'Наименование', 
                                  'Lot',  
                                  'Valid_until', 
                                  '1колонка', 
                                  '2колонка', 
                                  '3колонка', 
                                  'Себестоимость', 
                                  'Currency']]
columns_to_upload.rename(columns={
    'Lot'        : 'Лот',                               
    'Valid_until': 'Срок действия',                               
    'Currency'   : 'Валюта'          
}, inplace=True)



columns_to_upload = columns_to_upload.drop_duplicates(keep='first')



try:
    columns_to_upload.to_excel(f'{PRICE_PATH}\\totals\\upload.xlsx', index=False)
except Exception:
    print('Закройте файл upload.xlsx')


round(columns_to_upload.describe(),1)

,1колонка,2колонка,3колонка,Себестоимость
count,7708.0,7708.0,7708.0,7708.0
mean,46.8,43.7,40.6,31.2
std,60.1,56.1,52.1,40.0
min,4.5,4.2,3.9,3.0
25%,24.8,23.1,21.5,16.5
50%,37.4,34.9,32.4,24.9
75%,54.7,51.1,47.4,36.5
max,1276.3,1191.2,1106.1,850.9


In [3]:
from datetime import datetime

sign        = 'G2020'
incoterms   = 'FCA'
FCA_prepare =  price_calc[[ 'Код',                                   # готовим файл для загрузки
                            'Наименование', 
                            'Lot',  
                            'Valid_until', 
                            'Flavor_name', 
                            'FCA_price',                                                 
                            'Currency']]

FCA_prepare['Признак']   = sign
FCA_prepare['Клиент']    = sign
FCA_prepare['Период_id'] = FCA_prepare['Признак'] + FCA_prepare['Код']
FCA_prepare['Код_лота']  = FCA_prepare['Lot'].astype(str) + incoterms 
FCA_prepare['Дата предложения'] = datetime.today().date()

FCA_prepare   = FCA_prepare.loc[FCA_prepare['Наименование'] != FCA_prepare['Flavor_name']]

FCA_prepare   = FCA_prepare.loc[FCA_prepare['Код_лота']\
                                .str.contains('^((?!s).)*$')]         # рег. выражение not contains 's': ^((?!s).)*$

FCA_to_upload = FCA_prepare[['Код', 
                             'Valid_until', 
                             'Признак', 
                             'Период_id', 
                             'Currency', 
                             'Клиент', 
                             'Дата предложения', 
                             'Код_лота', 
                             'FCA_price']]

FCA_to_upload.to_excel(f'{PRICE_PATH}\\totals\\upload_FCA.xlsx', index=False)

round(FCA_to_upload.describe(),1)

,FCA_price
count,8269.0
mean,24.8
std,37.7
min,2.0
25%,13.1
50%,18.9
75%,27.4
max,690.5


# Поиск файла по коду

In [19]:
fca_df.loc[fca_df['Code']=='SY693933']

,Code,Flavor_name,Code_clean,Lot,Measure_unit,FCA_price,Currency,Valid_since,Valid_until,filename
5794,SY693933,Клубника 693933,693933,25,kg,40.90,EUR,2020-12-31,2020-12-31,allFCAs.xlsx
5795,SY693933,Клубника 693933,693933,50,kg,34.20,EUR,2020-12-31,2020-12-31,allFCAs.xlsx
5796,SY693933,Клубника 693933,693933,100,kg,30.10,EUR,2020-12-31,2020-12-31,allFCAs.xlsx
5797,SY693933,Клубника 693933,693933,200,kg,28.20,EUR,2020-12-31,2020-12-31,allFCAs.xlsx
5798,SY693933,Клубника 693933,693933,400,kg,26.50,EUR,2020-12-31,2020-12-31,allFCAs.xlsx
12695,SY693933,ERDBEER AROMA,693933,25,kg,40.90,EUR,2020-12-31,2020-12-31,SOT_30.01.20.xlsx
12696,SY693933,ERDBEER AROMA,693933,50,kg,34.20,EUR,2020-12-31,2020-12-31,SOT_30.01.20.xlsx
12697,SY693933,ERDBEER AROMA,693933,100,kg,30.10,EUR,2020-12-31,2020-12-31,SOT_30.01.20.xlsx
12698,SY693933,ERDBEER AROMA,693933,200,kg,28.20,EUR,2020-12-31,2020-12-31,SOT_30.01.20.xlsx
12699,SY693933,ERDBEER AROMA,693933,400,kg,26.50,EUR,2020-12-31,2020-12-31,SOT_30.01.20.xlsx
